# Chromacity Matrix Calculations

## Usage
See `USAGE.md` in the main repo for more instructions. 

## Inputs
This notebook takes in valuse from `csv/inputs.csv`.  Column headers are in the file, and all columns must be filled. 

## Outputs
4 output CSVs are generated. These are:

* `csv/matrix_XYZ.csv`. Contains both:
  * Msrc (expected source colourspace - what you want to convert your source from) RGB-to-XYZ 3x3 matrices
  * Mdst (expected destination/display colourspace - what you're going to look at with your eyes) XYZ-to-RGB 3x3 matrices
  * Created to allow people with fast graphics processing systems (GPU shaders, etc) the ability to stay in CIE XYZ colourspace and do further processing before returning to RGB

* `csv/matrix_sRGB.csv`, `csv/matrix_BT709.csv`, `csv/matrix_BT2020.csv`
  * These contain single 3x3 matrices for RGB to RGB conversion in a single matrix multiply, which can save precious calc on low power devices, FPGA chips, hardaware scalers, etc
  * Each file is pre-calculated to the given destination/display colourspace.  Inside are each of the source colourspaces used. 
  * Note that because these only apply to normalised (0.0-1.0) linear (EOTF/gamma removed) RGB, sRGB and BT709 are going to be identical
  * Likewise, the BT2020 matrices are colourspace/gamut only. You may choose to apply any EOTF/gamma you like, such as BT.1886 for BT.2020 SDR, or something like PQ/SMPTE-ST-2084 or HLG for BT.2020 HDR.  Don't forget to scale to appropriate 10/12 bit depth either.
 
## References
* All maths is from Bruce Lindbloom's site: http://www.brucelindbloom.com/
* If (x,y) reference values are included with a spec, they're put in the `inputs.csv` regardless of precision/accuracy (i.e.: unmodified from the era-matching spec, even in light of changes to Planck's c2 constant, etc)
* Specs are taken from official sources by preference (ITU/EBU/ARIB/SMPTE/CCIR/etc)
* If official sources aren't available, wikipedia as fallback
* If official specs don't exist, (e.g.: a whitepoint is listed as "D93"), then the values are calculated using the maths in this notebook, and things like Planck's c2 constant are taken into account.

In [1]:
import numpy as np
import pandas as pd
import csv
import datetime

In [2]:
## Define the colour science functions

# Planck adjusted c2
# Changes as late as 2018 affected constants in Planck's black body radiation calculations
# This affects only the CIE standard illuminant D family. Do not apply to CIE illuminants A/B/C/E/F.
# As such, slight adjustments need to be made to colour termperatures in the 1000s of K. 
# Multiply things like D65 "6500" by this value to get the more accurately adusted value. 
Planck_c2_adj = 1.438776877/1.4380

# K_to_xy takes in a colour temperature in Kelvin
# returns an array of the (x,y) co-ordinates of the white point in an array
# If you are using the CIE standard illluminant D series, adjust for Planck c2.
# Ref: http://www.brucelindbloom.com/Eqn_T_to_xy.html
def K_to_xy(T):
  if T < 7000:
      x = (-4.6070e+09/T**3)+(2.9678e+06/T**2)+(0.09911e+03/T)+0.244063
  else:
      x = (-2.0064e+09/T**3)+(1.9018e+6/T**2)+(0.24748e+03/T)+0.237040
  y = (-3.000*x*x)+(2.870*x)-0.275
  return np.array([x, y])

# xy_to_XYZ takes in a normalised Y=1 xyY (x,y) co-ordinate
# returns a CIE 1931 XYZ co-ordinate array
def xy_to_XYZ(coord):
    X = coord[0]/coord[1]
    Y = 1
    Z = (1-coord[0]-coord[1])/coord[1]
    return(np.array([X, Y, Z]))

# RGB_to_XYZ_matrix takes in two arrays:
# The first is the (x,y) co-ordinates of the R, G and B chromaticities .
# The second is the XYZ co-ordinate of the white point.
# It returns a 3x3 matrix that you can then apply to a linear RGB (no gamma/EOTF) pixel to convert it to XYZ respective colourspace.
# This is referrred to as "Msrc" above.
# Ref: http://www.brucelindbloom.com/Eqn_RGB_XYZ_Matrix.html
def RGB_to_XYZ_matrix(RGBxy, white_XYZ):
    xr = RGBxy[0]
    yr = RGBxy[1]
    xg = RGBxy[2]
    yg = RGBxy[3]
    xb = RGBxy[4]
    yb = RGBxy[5]
    Xr = xr/yr
    Yr = 1
    Zr = (1-xr-yr)/yr
    Xg = xg/yg
    Yg = 1
    Zg = (1-xg-yg)/yg
    Xb = xb/yb
    Yb = 1
    Zb = (1-xb-yb)/yb
    XYZrgb = np.array([[ Xr, Xg, Xb ],
                       [ Yr, Yg, Yb ],
                       [ Zr, Zg, Zb ]])
    XYZrgb_inv = np.linalg.inv(XYZrgb)
    white_XYZ_vert = np.reshape(white_XYZ, (3,1))
    Srgbvert = np.matmul(XYZrgb_inv, white_XYZ_vert)
    Srgb=np.reshape(Srgbvert, 3)
    return(Srgb*XYZrgb)

# To calculate an RGB_to_XYZ_matrix, perform a linear algebra matrix inversion of the RGB_to_XYZ_matrix function
# with respect to the currently configured colourspace and white point of your display/monitor/TV. 
# This will give you "Mdst" above.

In [3]:
# Common white points

std_ill_C_xy = K_to_xy(6774)
std_ill_C_XYZ = xy_to_XYZ(std_ill_C_xy)

std_ill_D65_xy = K_to_xy(6500*Planck_c2_adj)
std_ill_D65_XYZ = xy_to_XYZ(std_ill_D65_xy)

std_ill_D93_xy = K_to_xy(9300*Planck_c2_adj)
std_ill_D93_XYZ = xy_to_XYZ(std_ill_D93_xy)

np.set_printoptions(precision=32, suppress=True)

print("C, xy: ", std_ill_C_xy, "\n")
print("C, XYZ: ",std_ill_C_XYZ, "\n")

print("D65, xy: ",std_ill_D65_xy, "\n")
print("D65, XYZ: ",std_ill_D65_XYZ, "\n")

print("D93, xy: ",std_ill_D93_xy, "\n")
print("D93, XYZ: ",std_ill_D93_XYZ, "\n")


C, xy:  [0.3085489296804946 0.324928102162083 ] 

C, XYZ:  [0.9495913946112976 1.                 1.1280125225197994] 

D65, xy:  [0.3127219660174393  0.32912695838061345] 

D65, XYZ:  [0.9501560357015699 1.                 1.0881851713519297] 

D93, xy:  [0.28311093745916427 0.297072981780781  ] 

D93, XYZ:  [0.9530012987451018 1.                 1.4131748981125773] 



In [4]:
# Read inputs.csv from the csv directory for all input data
dfin=pd.read_csv('../csv/inputs.csv')
#dfin=pd.read_csv('csv/inputs.csv', index_col='col_id')


In [5]:
# Build the matrix for RGB to XYZ inputs and ouputs

# reset data lists
col_id_list = []
col_w_list = []
col_desc_list = []
eotf_list = []

Wx_list = []
Wy_list = []
Rx_list = []
Ry_list = []
Gx_list = []
Gy_list = []
Bx_list = []
By_list = []

Msrc0_list = []
Msrc1_list = []
Msrc2_list = []
Msrc3_list = []
Msrc4_list = []
Msrc5_list = []
Msrc6_list = []
Msrc7_list = []
Msrc8_list = []

Mdst0_list = []
Mdst1_list = []
Mdst2_list = []
Mdst3_list = []
Mdst4_list = []
Mdst5_list = []
Mdst6_list = []
Mdst7_list = []
Mdst8_list = []

dfin.reset_index()

for index, row in dfin.iterrows():
    white_XYZ = xy_to_XYZ([row['Wx'], row['Wy']])
    RGB_to_XYZ = RGB_to_XYZ_matrix([row['Rx'],row['Ry'],row['Gx'],row['Gy'],row['Bx'],row['By']], white_XYZ)
    XYZ_to_RGB = np.linalg.inv(RGB_to_XYZ)
    RGB_to_XYZ = np.reshape(RGB_to_XYZ, (1, 9)).tolist()
    XYZ_to_RGB = np.reshape(XYZ_to_RGB, (1, 9)).tolist()

    col_id_list.append(row['col_id'])
    col_w_list.append(row['col_w'])
    col_desc_list.append(row['col_desc'])
    eotf_list.append(row['eotf'])

    Wx_list.append(row['Wx'])
    Wy_list.append(row['Wy'])
    Rx_list.append(row['Rx'])
    Ry_list.append(row['Ry'])
    Gx_list.append(row['Gx'])
    Gy_list.append(row['Gy'])
    Bx_list.append(row['Bx'])
    By_list.append(row['By'])

    Msrc0_list.append(RGB_to_XYZ[0][0])
    Msrc1_list.append(RGB_to_XYZ[0][1])
    Msrc2_list.append(RGB_to_XYZ[0][2])
    Msrc3_list.append(RGB_to_XYZ[0][3])
    Msrc4_list.append(RGB_to_XYZ[0][4])
    Msrc5_list.append(RGB_to_XYZ[0][5])
    Msrc6_list.append(RGB_to_XYZ[0][6])
    Msrc7_list.append(RGB_to_XYZ[0][7])
    Msrc8_list.append(RGB_to_XYZ[0][8])

    Mdst0_list.append(XYZ_to_RGB[0][0])
    Mdst1_list.append(XYZ_to_RGB[0][1])
    Mdst2_list.append(XYZ_to_RGB[0][2])
    Mdst3_list.append(XYZ_to_RGB[0][3])
    Mdst4_list.append(XYZ_to_RGB[0][4])
    Mdst5_list.append(XYZ_to_RGB[0][5])
    Mdst6_list.append(XYZ_to_RGB[0][6])
    Mdst7_list.append(XYZ_to_RGB[0][7])
    Mdst8_list.append(XYZ_to_RGB[0][8])

    

matrix_XYZ_data = {'col_id': col_id_list,
     'col_w': col_w_list,
     'col_desc': col_desc_list,
     'eotf': eotf_list,
     'Wx': Wx_list,
     'Wy': Wy_list,
     'Rx': Rx_list,
     'Ry': Ry_list,
     'Gx': Gx_list,
     'Gy': Gy_list,
     'Bx': Bx_list,
     'By': By_list,
     'Msrc0': Msrc0_list,
     'Msrc1': Msrc1_list,
     'Msrc2': Msrc2_list,
     'Msrc3': Msrc3_list,
     'Msrc4': Msrc4_list,
     'Msrc5': Msrc5_list,
     'Msrc6': Msrc6_list,
     'Msrc7': Msrc7_list,
     'Msrc8': Msrc8_list,
     'Mdst0': Mdst0_list,
     'Mdst1': Mdst1_list,
     'Mdst2': Mdst2_list,
     'Mdst3': Mdst3_list,
     'Mdst4': Mdst4_list,
     'Mdst5': Mdst5_list,
     'Mdst6': Mdst6_list,
     'Mdst7': Mdst7_list,
     'Mdst8': Mdst8_list }

matrix_XYZ = pd.DataFrame(data=matrix_XYZ_data)
matrix_XYZ.to_csv('../csv/matrix_XYZ.csv', float_format='%0.16f', index=False)

In [6]:
# Built our XYZ_to_$display outputs

# sRGB
XYZ_to_sRGB_m = matrix_XYZ[matrix_XYZ["col_id"] == "sRGB" ]
XYZ_to_sRGB_m = XYZ_to_sRGB_m.to_dict(orient='list')
XYZ_to_sRGB_m = np.array([XYZ_to_sRGB_m['Mdst0'], XYZ_to_sRGB_m['Mdst1'], XYZ_to_sRGB_m['Mdst2'],
                          XYZ_to_sRGB_m['Mdst3'], XYZ_to_sRGB_m['Mdst4'], XYZ_to_sRGB_m['Mdst5'],
                          XYZ_to_sRGB_m['Mdst6'], XYZ_to_sRGB_m['Mdst7'], XYZ_to_sRGB_m['Mdst8'] ])
XYZ_to_sRGB_m = np.reshape(XYZ_to_sRGB_m, (3,3))

# BT709
XYZ_to_BT709_m = matrix_XYZ[matrix_XYZ["col_id"] == "BT709" ]
XYZ_to_BT709_m = XYZ_to_BT709_m.to_dict(orient='list')
XYZ_to_BT709_m = np.array([XYZ_to_BT709_m['Mdst0'], XYZ_to_BT709_m['Mdst1'], XYZ_to_BT709_m['Mdst2'],
                           XYZ_to_BT709_m['Mdst3'], XYZ_to_BT709_m['Mdst4'], XYZ_to_BT709_m['Mdst5'],
                           XYZ_to_BT709_m['Mdst6'], XYZ_to_BT709_m['Mdst7'], XYZ_to_BT709_m['Mdst8'] ])
XYZ_to_BT709_m = np.reshape(XYZ_to_BT709_m, (3,3))

# BT2020
XYZ_to_BT2020_m = matrix_XYZ[matrix_XYZ["col_id"] == "BT2020" ]
XYZ_to_BT2020_m = XYZ_to_BT2020_m.to_dict(orient='list')
XYZ_to_BT2020_m = np.array([XYZ_to_BT2020_m['Mdst0'], XYZ_to_BT2020_m['Mdst1'], XYZ_to_BT2020_m['Mdst2'],
                            XYZ_to_BT2020_m['Mdst3'], XYZ_to_BT2020_m['Mdst4'], XYZ_to_BT2020_m['Mdst5'],
                            XYZ_to_BT2020_m['Mdst6'], XYZ_to_BT2020_m['Mdst7'], XYZ_to_BT2020_m['Mdst8'] ])
XYZ_to_BT2020_m = np.reshape(XYZ_to_BT2020_m, (3,3))

#print(XYZ_to_sRGB)

In [7]:
# Build the matrix for RGB to RGB inputs and ouputs

# reset data lists
col_id_list = []
col_w_list = []
col_desc_list = []
eotf_list = []
Wx_list = []
Wy_list = []
Rx_list = []
Ry_list = []
Gx_list = []
Gy_list = []
Bx_list = []
By_list = []

M0_sRGB_list = []
M1_sRGB_list = []
M2_sRGB_list = []
M3_sRGB_list = []
M4_sRGB_list = []
M5_sRGB_list = []
M6_sRGB_list = []
M7_sRGB_list = []
M8_sRGB_list = []

M0_BT709_list = []
M1_BT709_list = []
M2_BT709_list = []
M3_BT709_list = []
M4_BT709_list = []
M5_BT709_list = []
M6_BT709_list = []
M7_BT709_list = []
M8_BT709_list = []

M0_BT2020_list = []
M1_BT2020_list = []
M2_BT2020_list = []
M3_BT2020_list = []
M4_BT2020_list = []
M5_BT2020_list = []
M6_BT2020_list = []
M7_BT2020_list = []
M8_BT2020_list = []

dfin.reset_index()

# grab our input csv and iterate
for index, row in dfin.iterrows():
    white_XYZ = xy_to_XYZ([row['Wx'], row['Wy']])
    RGB_to_XYZ = RGB_to_XYZ_matrix([row['Rx'],row['Ry'],row['Gx'],row['Gy'],row['Bx'],row['By']], white_XYZ)
    RGB_to_sRGB = np.matmul(XYZ_to_sRGB_m, RGB_to_XYZ)
    RGB_to_BT709 = np.matmul(XYZ_to_BT709_m, RGB_to_XYZ)
    RGB_to_BT2020 = np.matmul(XYZ_to_BT2020_m, RGB_to_XYZ)

    RGB_to_XYZ = np.reshape(RGB_to_XYZ, (1, 9)).tolist()
    RGB_to_sRGB = np.reshape(RGB_to_sRGB, (1, 9)).tolist()
    RGB_to_BT709 = np.reshape(RGB_to_BT709, (1, 9)).tolist()
    RGB_to_BT2020 = np.reshape(RGB_to_BT2020, (1, 9)).tolist()
    
    col_id_list.append(row['col_id'])
    col_w_list.append(row['col_w'])
    col_desc_list.append(row['col_desc'])
    eotf_list.append(row['eotf'])
    Wx_list.append(row['Wx'])
    Wy_list.append(row['Wy'])
    Rx_list.append(row['Rx'])
    Ry_list.append(row['Ry'])
    Gx_list.append(row['Gx'])
    Gy_list.append(row['Gy'])
    Bx_list.append(row['Bx'])
    By_list.append(row['By'])
    
    M0_sRGB_list.append(RGB_to_sRGB[0][0])
    M1_sRGB_list.append(RGB_to_sRGB[0][1])
    M2_sRGB_list.append(RGB_to_sRGB[0][2])
    M3_sRGB_list.append(RGB_to_sRGB[0][3])
    M4_sRGB_list.append(RGB_to_sRGB[0][4])
    M5_sRGB_list.append(RGB_to_sRGB[0][5])
    M6_sRGB_list.append(RGB_to_sRGB[0][6])
    M7_sRGB_list.append(RGB_to_sRGB[0][7])
    M8_sRGB_list.append(RGB_to_sRGB[0][8])

    M0_BT709_list.append(RGB_to_BT709[0][0])
    M1_BT709_list.append(RGB_to_BT709[0][1])
    M2_BT709_list.append(RGB_to_BT709[0][2])
    M3_BT709_list.append(RGB_to_BT709[0][3])
    M4_BT709_list.append(RGB_to_BT709[0][4])
    M5_BT709_list.append(RGB_to_BT709[0][5])
    M6_BT709_list.append(RGB_to_BT709[0][6])
    M7_BT709_list.append(RGB_to_BT709[0][7])
    M8_BT709_list.append(RGB_to_BT709[0][8])

    M0_BT2020_list.append(RGB_to_BT2020[0][0])
    M1_BT2020_list.append(RGB_to_BT2020[0][1])
    M2_BT2020_list.append(RGB_to_BT2020[0][2])
    M3_BT2020_list.append(RGB_to_BT2020[0][3])
    M4_BT2020_list.append(RGB_to_BT2020[0][4])
    M5_BT2020_list.append(RGB_to_BT2020[0][5])
    M6_BT2020_list.append(RGB_to_BT2020[0][6])
    M7_BT2020_list.append(RGB_to_BT2020[0][7])
    M8_BT2020_list.append(RGB_to_BT2020[0][8])

    
matrix_sRGB_data = {'col_id': col_id_list,
     'col_w': col_w_list,
     'col_desc': col_desc_list,
     'eotf': eotf_list,
     'Wx': Wx_list,
     'Wy': Wy_list,
     'Rx': Rx_list,
     'Ry': Ry_list,
     'Gx': Gx_list,
     'Gy': Gy_list,
     'Bx': Bx_list,
     'By': By_list,
     'M0': M0_sRGB_list,
     'M1': M1_sRGB_list,
     'M2': M2_sRGB_list,
     'M3': M3_sRGB_list,
     'M4': M4_sRGB_list,
     'M5': M5_sRGB_list,
     'M6': M6_sRGB_list,
     'M7': M7_sRGB_list,
     'M8': M8_sRGB_list }
matrix_sRGB = pd.DataFrame(data=matrix_sRGB_data)
matrix_sRGB.to_csv('../csv/matrix_sRGB.csv', float_format='%0.16f', index=False)

matrix_BT709_data = {'col_id': col_id_list,
     'col_w': col_w_list,
     'col_desc': col_desc_list,
     'eotf': eotf_list,
     'Wx': Wx_list,
     'Wy': Wy_list,
     'Rx': Rx_list,
     'Ry': Ry_list,
     'Gx': Gx_list,
     'Gy': Gy_list,
     'Bx': Bx_list,
     'By': By_list,
     'M0': M0_BT709_list,
     'M1': M1_BT709_list,
     'M2': M2_BT709_list,
     'M3': M3_BT709_list,
     'M4': M4_BT709_list,
     'M5': M5_BT709_list,
     'M6': M6_BT709_list,
     'M7': M7_BT709_list,
     'M8': M8_BT709_list }
matrix_BT709 = pd.DataFrame(data=matrix_BT709_data)
matrix_BT709.to_csv('../csv/matrix_BT709.csv', float_format='%0.16f', index=False)

matrix_BT2020_data = {'col_id': col_id_list,
     'col_w': col_w_list,
     'col_desc': col_desc_list,
     'eotf': eotf_list,
     'Wx': Wx_list,
     'Wy': Wy_list,
     'Rx': Rx_list,
     'Ry': Ry_list,
     'Gx': Gx_list,
     'Gy': Gy_list,
     'Bx': Bx_list,
     'By': By_list,
     'M0': M0_BT2020_list,
     'M1': M1_BT2020_list,
     'M2': M2_BT2020_list,
     'M3': M3_BT2020_list,
     'M4': M4_BT2020_list,
     'M5': M5_BT2020_list,
     'M6': M6_BT2020_list,
     'M7': M7_BT2020_list,
     'M8': M8_BT2020_list }
matrix_BT2020 = pd.DataFrame(data=matrix_BT2020_data)
matrix_BT2020.to_csv('../csv/matrix_BT2020.csv', float_format='%0.16f', index=False)

print("All finished and completed at: ", datetime.datetime.now())

All finished and completed at:  2023-11-04 16:41:34.333606
